In [1]:
# !pip install keras_cv_attention_models
#!pip install --upgrade tensorflow keras numpy
# !pip install numpy==1.24 tensorflow==2.17

# !pip uninstall tensorflow
# !pip install tensorflow==2.13.0  # or use the specific version you're working with


In [2]:
import os
from tensorflow import keras
import losses, train, GhostFaceNets
import tensorflow as tf
import keras_cv_attention_models

[WARNING] Setting TF_USE_LEGACY_KERAS=1. Make sure this is ahead of importing tensorflow or keras.


In [3]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print(tf.config.list_physical_devices('GPU'))

[]


In [4]:
# Remove the below for better accuracies and keep it for faster training
keras.mixed_precision.set_global_policy("mixed_float16")

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


GhostFaceNetV1

In [5]:
# # ms1m-retinaface-t1 (MS1MV3) dataset
# data_basic_path = 'datasets'  # Adjust this path as necessary
# data_path = data_basic_path + '_bin'  # Assuming you have your .bin file here

# # Paths to evaluation datasets
# eval_paths = [os.path.join(data_basic_path, 'dataset.bin')] 

# # # (MS1MV2) dataset
# # Main dataset folder (adjust the path if necessary)
# data_path = 'datasets/dataset'  # Path to your main dataset folder

# # Paths to evaluation datasets
# eval_paths = ['datasets/_bin/dataset.bin']


# data_path_ms1m_v3 = data_basic_path + '_bin'  # MS1M-RetinaFace
# data_path_ms1m_v2 = 'datasets/dataset'  # MS1M-ArcFace
# ms1m-retinaface-t1 (MS1MV3) dataset
data_basic_path = 'datasets/ms1m-retinaface-t1'
data_path = data_basic_path + '_112x112_folders'
eval_paths = [os.path.join(data_basic_path, ii) for ii in ['lfw.bin', 'cfp_fp.bin', 'agedb_30.bin']]

# (MS1MV2) dataset
data_path = 'datasets/faces_emore_112x112_folders'
eval_paths = ['datasets/faces_emore/lfw.bin', 'datasets/faces_emore/cfp_fp.bin', 'datasets/faces_emore/agedb_30.bin']

In [6]:
# #GhostFaceNetV1
# # Strides of 2
# basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
# basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
# basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

# # Strides of 1
# basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, scale=True, use_bias=True, strides=1)
# basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
# basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

#GhostFaceNetV1
# Strides of 2
basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, scale=True, use_bias=True, strides=1)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

/Users/niteshkumar/conda/envs/tf_env/lib/python3.9/site-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


>>>> Change BatchNormalization momentum and epsilon default value.
>>>> Convert ReLU: activation --> activation
>>>> Convert ReLU: activation_1 --> activation_1
>>>> Convert ReLU: activation_2 --> activation_2
>>>> Convert ReLU: activation_3 --> activation_3
>>>> Convert ReLU: activation_4 --> activation_4
>>>> Convert ReLU: activation_5 --> activation_5
>>>> Convert ReLU: activation_6 --> activation_6
>>>> Convert ReLU: activation_7 --> activation_7
>>>> Convert ReLU: activation_8 --> activation_8
>>>> Convert ReLU: activation_9 --> activation_9
>>>> Convert ReLU: activation_11 --> activation_11
>>>> Convert ReLU: activation_12 --> activation_12
>>>> Convert ReLU: activation_13 --> activation_13
>>>> Convert ReLU: activation_15 --> activation_15
>>>> Convert ReLU: activation_16 --> activation_16
>>>> Convert ReLU: activation_17 --> activation_17
>>>> Convert ReLU: activation_18 --> activation_18
>>>> Convert ReLU: activation_19 --> activation_19
>>>> Convert ReLU: activation_20 --> ac

In [7]:
# # Strides of 2
# tt = train.Train(data_path, eval_paths=eval_paths,
#     save_path='ghostnetv1_w1.3_s2.h5',
#     basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
#     batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

# # Strides of 1
# tt = train.Train(data_path, eval_paths=eval_paths,
#     save_path='ghostnetv1_w1.3_s1.h5',
#     basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
#     batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

# Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s2.weights.h5',  # Updated here
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

# Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s1.weights.h5',  # Updated here
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)


>>>> L2 regularizer value from basic_model: 0.00025
abc edvhwjdbf :
abc edvhwjdbf :
abc edvhwjdbf :
>>>> L2 regularizer value from basic_model: 0.00025
abc edvhwjdbf :
abc edvhwjdbf :
abc edvhwjdbf :


In [8]:
# optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
# sch = [
#     {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
#     {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50, "optimizer": optimizer},
# ]
# tt.train(sch, 0)
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": "SGD"},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50, "optimizer": "SGD"},
]
tt.train(sch, 0)

>>>> Init type by loss function name...
>>>> Train arcface...
>>>> Init softmax dataset...
>>>> reloaded from dataset backup: faces_emore_112x112_folders_shuffle.npz
>>>> Image length: 100, Image class length: 100, classes: 10
>>>> Add L2 regularizer to model output layer, output_weight_decay = 0.000500
>>>> Add arcface layer, arc_kwargs={'loss_top_k': 1, 'append_norm': False, 'partial_fc_split': 0, 'name': 'arcface'}, vpl_kwargs={'vpl_lambda': 0.15, 'start_iters': 0, 'allowed_delta': 200}...
>>>> loss_weights: {'arcface': 1}

Epoch 1: saving model to checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5


2025-01-13 00:35:44.538905: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/niteshkumar/conda/envs/tf_env/lib/python3.9/contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


0/1 ━━━━━━━━━━━━━━━━━━━━ 15s 0s/step


>>>> Train arcface DONE!!! epochs = [0], model.stop_training = False
>>>> My history:
{
}
>>>> Saving latest basic model to: checkpoints/ghostnetv1_w1.3_s1.weights_basic_model_latest.h5
>>>> Init type by loss function name...
>>>> Train arcface...
>>>> Add L2 regularizer to model output layer, output_weight_decay = 0.000500
>>>> Will NOT change model output layer.
>>>> loss_weights: {'arcface': 1}
Epoch 2/51

Epoch 2: saving model to checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5


2025-01-13 00:35:59.131994: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


...


Epoch 17: saving model to checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5
0/1 ━━━━━━━━━━━━━━━━━━━━ 2s 0s/step
Epoch 18/51

Epoch 18: saving model to checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5
0/1 ━━━━━━━━━━━━━━━━━━━━ 2s 0s/step
Epoch 19/51

Epoch 19: saving model to checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5
0/1 ━━━━━━━━━━━━━━━━━━━━ 2s 0s/step
Epoch 20/51

Epoch 20: saving model to checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5
0/1 ━━━━━━━━━━━━━━━━━━━━ 2s 0s/step
Epoch 21/51

Epoch 21: saving model to checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5
0/1 ━━━━━━━━━━━━━━━━━━━━ 2s 0s/step
Epoch 22/51

Epoch 22: saving model to checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5
0/1 ━━━━━━━━━━━━━━━━━━━━ 2s 0s/step
Epoch 23/51

Epoch 23: saving model to checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5
0/1 ━━━━━━━━━━━━━━━━━━━━ 2s 0s/step
Epoch 24/51

Epoch 24: saving model to checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5
0/1 ━━━━━━━━━━━━━━━━━━━━ 2s 0s/step
Epoch 25/51


2025-01-13 00:37:10.173315: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


Epoch 33/51

Epoch 33: saving model to checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5
0/1 ━━━━━━━━━━━━━━━━━━━━ 2s 0s/step
Epoch 34/51

Epoch 34: saving model to checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5
0/1 ━━━━━━━━━━━━━━━━━━━━ 2s 0s/step
Epoch 35/51

Epoch 35: saving model to checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5
0/1 ━━━━━━━━━━━━━━━━━━━━ 2s 0s/step
Epoch 36/51

Epoch 36: saving model to checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5
0/1 ━━━━━━━━━━━━━━━━━━━━ 3s 0s/step
Epoch 37/51

Epoch 37: saving model to checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5
0/1 ━━━━━━━━━━━━━━━━━━━━ 2s 0s/step
Epoch 38/51

Epoch 38: saving model to checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5
0/1 ━━━━━━━━━━━━━━━━━━━━ 2s 0s/step
Epoch 39/51

Epoch 39: saving model to checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5
0/1 ━━━━━━━━━━━━━━━━━━━━ 2s 0s/step
Epoch 40/51

Epoch 40: saving model to checkpoints/ghostnetv1_w1.3_s1.weights.weights.h5
0/1 ━━━━━━━━━━━━━━━━━━━━ 2s 0s/step


>>>> Train arcface DONE!!! epochs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50], model.stop_training = False
>>>> My history:
{
}
>>>> Saving latest basic model to: checkpoints/ghostnetv1_w1.3_s1.weights_basic_model_latest.h5


51

GhostFaceNetV2

In [9]:
# ms1m-retinaface-t1 (MS1MV3) dataset
data_basic_path = 'datasets/ms1m-retinaface-t1'
data_path = data_basic_path + '_112x112_folders'
eval_paths = [os.path.join(data_basic_path, ii) for ii in ['lfw.bin', 'cfp_fp.bin', 'agedb_30.bin']]

# (MS1MV2) dataset
data_path = 'datasets/faces_emore_112x112_folders'
eval_paths = ['datasets/faces_emore/lfw.bin', 'datasets/faces_emore/cfp_fp.bin', 'datasets/faces_emore/agedb_30.bin']

In [10]:
#GhostFaceNetV2
# Strides of 2
basic_model = GhostFaceNets.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, stem_strides=1)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

>>>> Change BatchNormalization momentum and epsilon default value.


TypeError: Layer tf.nn.convolution was passed non-JSON-serializable arguments. Arguments had types: {'filters': <class 'keras.src.backend.tensorflow.core.Variable'>, 'strides': [<class 'int'>, <class 'int'>], 'padding': <class 'str'>, 'data_format': <class 'str'>, 'dilations': (<class 'int'>, <class 'int'>)}. They cannot be serialized out when saving the model.

In [10]:
#Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s2.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

#Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s1.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

>>>> L2 regularizer value from basic_model: 0.00025
Shape of bins: (3,)
First few elements in bins: ('datasets/faces_emore_112x112_folders/00007/7_6.jpg', 'datasets/faces_emore_112x112_folders/00004/4_5.jpg', False)
Shape of bins: (3,)
First few elements in bins: ('datasets/faces_emore_112x112_folders/00007/7_6.jpg', 'datasets/faces_emore_112x112_folders/00004/4_5.jpg', False)
Shape of bins: (3,)
First few elements in bins: ('datasets/faces_emore_112x112_folders/00007/7_6.jpg', 'datasets/faces_emore_112x112_folders/00004/4_5.jpg', False)
>>>> L2 regularizer value from basic_model: 0.00025
Shape of bins: (3,)
First few elements in bins: ('datasets/faces_emore_112x112_folders/00007/7_6.jpg', 'datasets/faces_emore_112x112_folders/00004/4_5.jpg', False)
Shape of bins: (3,)
First few elements in bins: ('datasets/faces_emore_112x112_folders/00007/7_6.jpg', 'datasets/faces_emore_112x112_folders/00004/4_5.jpg', False)
Shape of bins: (3,)
First few elements in bins: ('datasets/faces_emore_112x1

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50},
]
tt.train(sch, 0)

CosFaceLoss

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.CosFaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.CosFaceLoss(scale=64), "epoch": 50},
]
tt.train(sch, 0)

Subcenter ArcFace Loss

In [ ]:
# ms1m-retinaface-t1 (MS1MV3) dataset
data_basic_path = 'datasets/ms1m-retinaface-t1'
data_path = data_basic_path + '_112x112_folders'
eval_paths = [os.path.join(data_basic_path, ii) for ii in ['lfw.bin', 'cfp_fp.bin', 'agedb_30.bin']]

# (MS1MV2) dataset
data_path = 'datasets/faces_emore_112x112_folders'
eval_paths = ['datasets/faces_emore/lfw.bin', 'datasets/faces_emore/cfp_fp.bin', 'datasets/faces_emore/agedb_30.bin']

In [8]:
# GhostFaceNetV1
# Strides of 2
basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, scale=True, use_bias=True, strides=1)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

>>>> Change BatchNormalization momentum and epsilon default value.
>>>> Convert ReLU: activation_96 --> activation_96
>>>> Convert ReLU: activation_97 --> activation_97
>>>> Convert ReLU: activation_98 --> activation_98
>>>> Convert ReLU: activation_99 --> activation_99
>>>> Convert ReLU: activation_100 --> activation_100
>>>> Convert ReLU: activation_101 --> activation_101
>>>> Convert ReLU: activation_102 --> activation_102
>>>> Convert ReLU: activation_103 --> activation_103
>>>> Convert ReLU: activation_104 --> activation_104
>>>> Convert ReLU: activation_105 --> activation_105
>>>> Convert ReLU: activation_107 --> activation_107
>>>> Convert ReLU: activation_108 --> activation_108
>>>> Convert ReLU: activation_109 --> activation_109
>>>> Convert ReLU: activation_111 --> activation_111
>>>> Convert ReLU: activation_112 --> activation_112
>>>> Convert ReLU: activation_113 --> activation_113
>>>> Convert ReLU: activation_114 --> activation_114
>>>> Convert ReLU: activation_115 --> ac

In [ ]:
# GhostFaceNetV2
# Strides of 2
basic_model = GhostFaceNets.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, stem_strides=1)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

In [9]:
#GhostFaceNetV1
#Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s2_topk.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

#Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s1_topk.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

>>>> L2 regularizer value from basic_model: 0.00025
Shape of bins: (3,)
First few elements in bins: ('datasets/faces_emore_112x112_folders/00007/7_6.jpg', 'datasets/faces_emore_112x112_folders/00004/4_5.jpg', False)
Shape of bins: (3,)
First few elements in bins: ('datasets/faces_emore_112x112_folders/00007/7_6.jpg', 'datasets/faces_emore_112x112_folders/00004/4_5.jpg', False)
Shape of bins: (3,)
First few elements in bins: ('datasets/faces_emore_112x112_folders/00007/7_6.jpg', 'datasets/faces_emore_112x112_folders/00004/4_5.jpg', False)
>>>> L2 regularizer value from basic_model: 0.00025
Shape of bins: (3,)
First few elements in bins: ('datasets/faces_emore_112x112_folders/00007/7_6.jpg', 'datasets/faces_emore_112x112_folders/00004/4_5.jpg', False)
Shape of bins: (3,)
First few elements in bins: ('datasets/faces_emore_112x112_folders/00007/7_6.jpg', 'datasets/faces_emore_112x112_folders/00004/4_5.jpg', False)
Shape of bins: (3,)
First few elements in bins: ('datasets/faces_emore_112x1

In [ ]:
#GhostFaceNetV2
#Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s2_topk.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

#Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s1_topk.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

In [10]:
""" First, Train with `lossTopK = 3` """
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer, "lossTopK": 3},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50, "lossTopK": 3},
]
tt.train(sch, 0)

>>>> Init type by loss function name...
>>>> Train arcface...
>>>> Init softmax dataset...
>>>> reloaded from dataset backup: faces_emore_112x112_folders_shuffle.npz
>>>> Image length: 100, Image class length: 100, classes: 10
>>>> Add L2 regularizer to model output layer, output_weight_decay = 0.000500
>>>> Add arcface layer, arc_kwargs={'loss_top_k': 3, 'append_norm': False, 'partial_fc_split': 0, 'name': 'arcface'}, vpl_kwargs={'vpl_lambda': 0.15, 'start_iters': 0, 'allowed_delta': 200}...
>>>> loss_weights: {'arcface': 1}


ValueError: Could not interpret optimizer identifier: <tf_keras.src.optimizers.sgd.SGD object at 0x157c43e50>

In [ ]:
""" Then drop non-dominant subcenters and high-confident noisy data, which is `>75 degrees` """
import data_drop_top_k
# data_drop_top_k.data_drop_top_k('./checkpoints/TT_mobilenet_topk_bs256.h5', '/datasets/faces_casia_112x112_folders/', limit=20)
new_data_path = data_drop_top_k.data_drop_top_k(tt.model, tt.data_path)

In [ ]:
""" Train with the new dataset again, this time `lossTopK = 1` """
tt.reset_dataset(new_data_path)

At this point you can continue training the model or start training again and considering the generated dataset as a new dataset

In [ ]:
# Continue training
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50},
]
tt.train(sch, 0)

In [ ]:
# Train the models from scatch using the new generated dataset

# New Cleaned Dataset
data_path = 'Path_To_New_Generated_Dataset'
eval_paths = ['datasets/faces_emore/lfw.bin', 'datasets/faces_emore/cfp_fp.bin', 'datasets/faces_emore/agedb_30.bin']

In [ ]:
# GhostFaceNetV1
# Strides of 2
basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, scale=True, use_bias=True, strides=1)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

In [ ]:
# GhostFaceNetV2
# Strides of 2
basic_model = GhostFaceNets.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, stem_strides=1)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

In [ ]:
# GhostFaceNetV1
# Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s2.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

# Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s1.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

In [ ]:
# GhostFaceNetV2
#Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s2.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

#Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s1.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50},
]
tt.train(sch, 0)